In [1]:
# importando pacotes
import matplotlib
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

### Segunda Questão

### Objetivo 

Este projeto tem como objetivo calcular similaridade entre nomes dos produtos. Para isso, vamos calcular a similaridade entre cada linha do items_titles_test e todas as linhas do items_titles e guardar quais pares tiveram maior similaridade. Ao fim teremos um dataset com 10.000 linhas que será o equivalente ao items_titles_test e seu respectivo par com maior similaridade do items_titles. 

### Entendimento dos dados 

Os dados encontram-se em csv (items_titles.csv e items_titles_test.csv) 

In [2]:
# puxando os dados items_titles
df_items_titles = pd.read_csv("arquivos/items_titles.csv", sep=",")

print("Dimensão da base items_titles:", df_items_titles.shape)
print("Quantidade de valores distintos:", df_items_titles.nunique()[0])
df_items_titles.head(3)

Dimensão da base items_titles: (30000, 1)
Quantidade de valores distintos: 30000


,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...


In [3]:
# puxando os dados items_titles_test
df_items_titles_test = pd.read_csv("arquivos/items_titles_test.csv", sep=",")

print("Dimensão da base items_titles_test:", df_items_titles_test.shape)
print("Quantidade de valores distintos:", df_items_titles_test.nunique()[0])
df_items_titles_test.head(3)

Dimensão da base items_titles_test: (10000, 1)
Quantidade de valores distintos: 10000


,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2


Quantidade de valores idênticos entre as duas bases:

In [4]:
column1 = set(df_items_titles['ITE_ITEM_TITLE'])
column2 = set(df_items_titles_test['ITE_ITEM_TITLE'])

print("Quantidade de valores idênticos:", len(column1.intersection(column2)))

Quantidade de valores idênticos: 1493


### Similaridade

Para calcular a similaridade entre dois nomes podemos utilizar calculos como distância de Levenshtein ou distância de Jaccard, os calculos possuem algumas diferenças, dentre elas a eficiencia computacional, sendo Jaccard mais eficiente computacionalmente. 

Vamos analisar alguns exemplos práticos entre os dois cálculos e o tempo gasto computacionalmente (com uma amostra) para que seja possível escolher o que melhor se adapta ao desafio em questão. 

In [5]:
# definindo função para cálculo de distância de Levenshtein 
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)

    for i, c1 in enumerate(s1):
        current_row = [i + 1]

        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        previous_row = current_row

    return previous_row[-1]

# definindo função para cálculo de similaridade de Levenshtein 
# A similaridade de Levenshtein é calculada como o inverso da distância de Levenshtein. 
# Esta distância é o número mínimo de operações necessárias para transformar uma string na outra.
def levenshtein_similarity(s1, s2):
    distance = levenshtein_distance(s1, s2)
    similarity = 1 / (1 + distance)
    return similarity

# definindo função para cálculo de similaridade de Jaccard 
def jaccard_similarity(s1, s2):
    # Converte os textos em conjuntos de palavras
    set1 = set(s1.split())
    set2 = set(s2.split())
    
    # Calcula a interseção e a união dos conjuntos
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    # Calcula a similaridade de Jaccard
    similarity = intersection / union if union != 0 else 0
    
    return similarity

In [6]:
# apenas conferindo se faz sentido 
print(levenshtein_similarity('a','a'))
print(jaccard_similarity('a','a'))

1.0
1.0


In [7]:
# apenas conferindo se faz sentido (seria interessante calcular utilizando .lower() nas bases)
print(levenshtein_similarity('a','A'))
print(jaccard_similarity('a','A'))

0.5
0.0


In [8]:
# apenas conferindo se faz sentido
print(levenshtein_similarity('a','a.'))
print(jaccard_similarity('a','a.'))

0.5
0.0


No caso de 'a' e 'a.', a distância de Levenshtein é 0.5 porque você precisa adicionar ou remover um caractere ('.') para transformar uma string na outra. Portanto, a similaridade de Levenshtein é 1 - 0.5 = 0.5

A similaridade de Jaccard é então calculada como a razão entre o tamanho da interseção dos conjuntos e o tamanho da união dos conjuntos. Como a interseção é vazia e a união contém apenas um caractere ou palavras (a), a similaridade de Jaccard é 0.